In [ ]:
import os
import pandas as pd
from PDFTextProcessor import PDFTextProcessor
from AnsweringClass import AnsweringClass

# Initialize AnsweringClass and set the books directory
ans = AnsweringClass()
books_dir = "/home/saurabh/Desktop/Enigmatica /books"
print(os.listdir(books_dir))
# Create an empty DataFrame to store data from all books
all_books_df = pd.DataFrame()

# Iterate over all PDF files in the directory
for file_name in os.listdir(books_dir):
    print(file_name)
    if file_name.endswith(".pdf"):  # Process only PDF files
        pdf_path = os.path.join(books_dir, file_name)
        base_name = os.path.splitext(file_name)[0]  # Get the base name without extension
        # Define file paths for output and cleaned text
        output_txt_path = "output_text.txt"
        cleaned_txt_path = "cleaned_text.txt"

        # Initialize PDFTextProcessor for each book
        processor = PDFTextProcessor(
            pdf_path=pdf_path,
            output_txt_path=output_txt_path,
            cleaned_txt_path=cleaned_txt_path
        )

        # Step 1: Extract and clean text from the PDF
        processor.extract_text()
        processor.clean_text_file()

        # Step 2: Process the cleaned text into paragraphs and create a DataFrame
        df = processor.process_text_to_dataframe(cleaned_txt_path, output_csv_path="paragraphs.csv")
        
        # Step 3: Add embeddings for the paragraphs
        df["embedding"] = df["paragraph"].apply(lambda x: ans.get_sentence_embedding(x))

        # Add the book source (filename) for tracking
        df["Source"] = base_name

        # Append the data from this book to the overall DataFrame
        all_books_df = pd.concat([all_books_df, df], ignore_index=True)

# After processing all books, save the combined DataFrame as a CSV file
all_books_df.to_csv("all_books_paragraphs.csv", index=False)

# Optionally print or check the combined DataFrame
print("Combined DataFrame saved to all_books_paragraphs.csv")


In [ ]:
import pandas as pd
from PDFTextProcessor import PDFTextProcessor
from AnsweringClass import AnsweringClass


In [ ]:
df1 = pd.read_csv("all_books_paragraphs.csv")

In [ ]:
df1.iloc[0]

In [ ]:
df1.head()

In [ ]:
df = df1.drop('embedding', axis=1)
df = df.drop("Source",axis=1)
df.columns

In [ ]:
df["embedding"] = df["paragraph"].apply(lambda x: ans.get_sentence_embedding(x))
df

In [ ]:
question = input()

# Generate the embedding for the question
question_embedding = ans.get_sentence_embedding(question)

ansList = ans.get_top_matches(question,df,embedding_column="embedding")
answer = ""
for tup in ansList:
    answer = answer+tup[0]

In [ ]:
print(answer)

Andriy Burkov'sPreface Let’s start by telling the truth: machines don’t learn. What a typical “learning machine” does, is ﬁnding a mathematical formula, which, when applied to a collection of inputs (called “training data”), produces the desired outputs. This mathematical formula also generates the correct outputs for most other inputs (distinct from the training data) on the condition that those inputs come from the same or a similar statistical distribution as the one the training data was drawn from.The goal of machine learning is to develop methods that can automatically detect patterns in data, and then to use the uncovered patterns to predict future data or other outcomes of interest. Machine learning is thus closely related to the ﬁelds of statistics and data mining, but differs slightly in terms of its emphasis and terminology.This includes supervised learning techniques that compare known combinations of input and output variables to discern patterns and make predictions, and reinforcement learning which randomly trials a massive number of input variables to produce a desired output. Another machine learning technique, called unsupervised learning, generates predictions based on the analysis of input variables with no known target output.We are also grateful for the members of a book reading club in Jerusalem that have carefully read and constructively criticized every line of vani, Aharon Birnbaum, Alon Cohen, Alon Gonen, Roi Livni, Ofer Meshi, Dan Rosenbaum, Dana Rubinstein, Shahar Somin, Alon Vinnikov, and Yoav Wald. We would also like to thank Gal Elidan, Amir Globerson, Nika Haghtalab, Shie Mannor, Amnon Shashua, Nati Srebro, and Ruth Urner for helpful discussions.Introduction The subject of this book is automated learning, or, as we will more often call it, Machine Learning (ML).A Plain English Introduction Oliver TheobaldThird Edition All rights reserved. No part of this publication may be reproduced, photocopying, recording, or other electronic or mechanical methods, without the prior written permission of the publisher, except in the case of brief quotations embodied in critical reviews and certain other Edited by Jeremy Pedersen and Red to Black Editing’s Christopher For feedback, print quality issues, media contact, omissions or errors regarding this book, please contact the author at oliver.theobald@scatterplotpress.comTABLE OF CONTENTS WHAT IS MACHINE LEARNING?

In [ ]:
import pandas as pd
import numpy as np
import re

In [ ]:
df = pd.read_csv("all_books_paragraphs.csv")


In [ ]:
import pandas as pd
import numpy as np
import psycopg2
from psycopg2.extras import execute_values


# Set up your connection and cursor
conn = psycopg2.connect(
    dbname="mydatabase",
    user="postgres",
    password="Saurabh@1",
    host="localhost",  # or your host
    port="5432"        # default PostgreSQL port
)
cursor = conn.cursor()

# Prepare the data for insertion
insert_data = [(text, [embedding.replace(" ",",")]) for text, embedding in zip(df['paragraph'], df['embedding'])]
print(insert_data)
# # Insert data into PostgreSQL table
# insert_query = """
# INSERT INTO embeddings (text, embedding)
# VALUES (%s, %s::vector(384));  -- Assuming vector length is 384, adjust as needed
# """

# execute_values(cursor, insert_query, insert_data)

# # Commit the transaction and close the connection
# conn.commit()
# cursor.close()
# conn.close()


In [ ]:
insert_data